In [ ]:
pip install opencv-python

In [1]:

import cv2
import numpy as np
keep_percent = 0.75
thresh = 250.0
nfeat = 2**15 - 1
widen = 2500
abort = 0
start_time = 0.0
# Load the images
# color print function
import time
def track_time():
    return time.time()

    
def output_time():
    curr_time = time.time()
    e = curr_time - start_time
    print(f"Program has been running for {e:.2f} seconds.")
        
def cprint(color, text):
    colors = {
        'red': '\033[91m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'blue': '\033[94m',
        'magenta': '\033[95m',
        'cyan': '\033[96m',
        'white': '\033[97m',
        'reset': '\033[0m'
    }
    print(f"{colors.get(color, colors['reset'])}{text}{colors['reset']}")

def rot_matrix(f):
    return np.array([[np.cos(f), -np.sin(f)], [np.sin(f), np.cos(f)]])

def aff_matrix(s, t, mx):
    return np.array([[s * mx[0][0], s * mx[0][1], t[0]], [s * mx[1][0], s * mx[1][1], t[1]]])

def read_img(n):
    s1 = '2023_09_01_SonyRX1RM2_g201b20538_f001_'
    s3 = '.JPG'
    s = str(n)
    l = len(s)
    s2 = '0' * (4-l) + s
    return cv2.imread('./folder2/' + s1 + s2 + s3)

def crop_img(image_1):
    mask = np.any(image_1 != 0, axis=-1)
    coords = np.argwhere(mask)
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0) + 1
    return y_min, x_min, image_1[y_min:y_max, x_min:x_max]
    
def image_get_rot(image_base, image_fit):
     
    # Convert images to grayscale
    gray1 = cv2.cvtColor(image_base, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image_fit, cv2.COLOR_BGR2GRAY)
    
    # Detect ORB keypoints and descriptors
    orb = cv2.ORB_create(nfeatures = nfeat)
    keypoints1, descriptors1 = orb.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(gray2, None)

    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
 
    # Match descriptors.
    matches = bf.match(descriptors1, descriptors2)
    
    # Sort matches by distance
    matches = sorted(matches, key=lambda x: x.distance)

    keep = int(len(matches) * keep_percent)
    matches = matches[:keep]
    

    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    print('bijection part of transform: ', len(matches), '/', nfeat, sep = '')

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find the rotation transformation matrix
    matrix, _ = cv2.estimateAffinePartial2D(points2, points1, method=cv2.RANSAC, ransacReprojThreshold = thresh, maxIters = 20000, confidence = 0.99999)
    inn = np.count_nonzero(_ == 1)
    print('inner part of transform: ', inn, '/', nfeat, sep = '')
    if(inn/nfeat < 0.01):
        cprint('red', 'Successful warp was not found')
        abort = 1
    print(matrix)
    sn = -matrix[0, 1]
    cn = matrix[0, 0]
    fi = np.arctan(sn/cn)
    print('angle is', fi)
    sc = cn/np.cos(fi)
    print('scale is', cn/np.cos(fi))
    tr = [matrix[0][2], matrix[1][2]]
    print('translation value is:', tr)
    return fi, sc, tr


def image_widen(image_w, w_val):
    height, width, _ = image_w.shape
    #5304x7952
    canvas_height = height + w_val
    canvas_width = width + w_val
    green_canvas = np.zeros((canvas_height, canvas_width, 4), dtype=np.uint8)
    green_canvas[:] = (0, 0, 0, 0)  # Set the canvas to green
    y_offset = (canvas_height - height) // 2
    x_offset = (canvas_width - width) // 2
    alpha_image_w = np.zeros((height, width, 4), dtype=np.uint8)
    if(image_w.shape[2] == 3):
        alpha_image_w[:,:,0:3] = image_w
        alpha_image_w[:,:,3] = 255
    else:
        alpha_image_w = image_w
    green_canvas[y_offset:y_offset + height, x_offset:x_offset + width] = alpha_image_w
    return green_canvas

def image_overlay(image_bb, image_ol):
    h, w, _ = image_ol.shape
    print("overlay shape:", h, w)
    # Define the region of interest (ROI) on the base image
    roi = image_bb[0:h, 0:w]

    # Create a mask of the overlay image and its inverse mask
    result_gray = cv2.cvtColor(image_ol, cv2.COLOR_BGRA2GRAY)
    _, mask = cv2.threshold(result_gray, 1, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)

    # Black-out the area of the overlay in the ROI
    image1_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

    # Take only the region of the overlay from the overlay image
    result_fg = cv2.bitwise_and(image_ol, image_ol, mask=mask)
    
    # Put the overlay in the ROI and modify the main image
    dst = cv2.add(image1_bg, result_fg)
    image_bb[0:h, 0:w] = dst
    return image_bb

def image_intersect(image1, image2):
    
    # Get the angle, scale and translations of the axes to fit image2 on image1
    fl, sl, tl = image_get_rot(image1, image2)

    mxl = rot_matrix(fl)
    matrix = aff_matrix(sl, tl, mxl)
    
    print(fl, sl, tl, mxl)
    print(matrix)
    
    
    # Apply the rotation transformation
    result = cv2.warpAffine(image2, matrix, (image2.shape[1], image2.shape[0]))
    #cv2.imwrite('output.png', result) --Enable to see warped image

    image1 = image_overlay(image1, result)
    
    # Save the overlay image
    #cv2.imwrite('result_target.png', image1) --Enable to see overlayed image
    
    print('Assembled target with pastesq')
    
    return fl, sl, tl, image1
start_time = track_time()

abort = 0
base = read_img(3)
center_h, center_w, _ = base.shape
center_h = center_h // 2
center_w = center_w // 2
print(center_h, center_w)
for i in range (4, 11):
    output_time()
    print('Assembling image', i, '...')
    paste = read_img(i)
    u = 4
    ppp = u * widen
    center_h += ppp
    center_w += ppp
    paste = image_widen(paste, 2 * ppp)
    p_center_h, p_center_w, _ = paste.shape
    p_center_h = p_center_h // 2
    p_center_w = p_center_w // 2
    pastesq = paste[p_center_h  - ppp: p_center_h + ppp,p_center_w - ppp: p_center_w + ppp,:]
    base = image_widen(base, 2 * ppp)
    target = base[center_h  - ppp: center_h + ppp,center_w - ppp: center_w + ppp,:]
    cv2.imwrite('target.png', target) #enable to see target of where algorithm should find place for new image
    cv2.imwrite('pastesq.png', pastesq) #enable to see paste of where algorithm should find place for new image
    fr, sr, tr, aba = image_intersect(target, pastesq)
    base[center_h - ppp: center_h + ppp, center_w - ppp: center_w + ppp] = aba
    vepp = np.array([ppp, ppp])
    delta_vepp = np.dot(rot_matrix(fr), vepp) * sr - vepp
    tr = tr + delta_vepp
    ww = int(tr[0])
    hh = int(tr[1])
    center_h += hh
    center_w += ww
    base[center_h-100:center_h+100, center_w-100:center_w+100,:] = 255
    h_min, w_min, base = crop_img(base)
    center_h -= h_min
    center_w -= w_min
    base[center_h-50:center_h+50, center_w-50:center_w+50] = [0, 0, 0, 255]
    cv2.imwrite('base.png', base)
    print('Image', i, 'has been assembled')
    
 
    

ModuleNotFoundError: No module named 'cv2'

In [19]:










np.sqrt(5304**2 + 7952**2)

9558.594038874127

In [22]:
5304 * 1.4

7425.599999999999

In [30]:
import numpy as np
import cv2

# Define the transformation matrix
mat = np.array([[1, 0, 100], [0, 1, 50],[0, 0, 1]], dtype=np.float32)

# Define the point to be transformed
center_h, center_w = 200, 300
points_to_be_transformed = np.array([[[center_h, center_w]]], dtype=np.float32)

# Perform the perspective transformation
transformed_points = cv2.perspectiveTransform(points_to_be_transformed, mat)

print("Transformed Points:", transformed_points)

Transformed Points: [[[300. 350.]]]


In [1]:
import numpy as np
import cv2
angle = np.pi
s = 1.0
img = read_img(3)
h, w, _ = img.shape
print(h, w)
mat = np.array([[s * np.cos(angle), -s * np.sin(angle), w], [s * np.sin(angle), s * np.cos(angle), h]], dtype=np.float32)
result = cv2.warpAffine(img, mat, (img.shape[1], img.shape[0]))
cv2.imwrite('result_target.png', result)
small = cv2.resize(image, (0,0), fx=0.5, fy=0.5) 


NameError: name 'read_img' is not defined